# <center>Rapport de TP - implémentation de Hadoop MapReduce "from scratch" en Java"</center>

### <font color="red">Etape 1</font> : Faire un programme séquentiel non parallélisé qui compte le nombre d'occurrences des mots dans un fichier. 

In [9]:
import java.io.IOException;
import java.nio.file.Files;
import java.nio.file.Paths;
import java.util.ArrayList;
import java.util.Collections;
import java.util.Comparator;
import java.util.HashMap;
import java.util.LinkedHashMap;
import java.util.Map;
import java.util.Map.Entry;

import static java.util.stream.Collectors.*;

public class WordsCount{
    
    public static ArrayList<String> read_file(String filename){
        ArrayList<String> lines = new ArrayList<>();
        try {
            lines = (ArrayList<String>) Files.readAllLines(Paths.get(filename));
        } catch (IOException e) {
            System.out.println("Problem when loading file");
            return null;
        }
        return lines;
    }
    
    public static ArrayList<String[]> tokenize(ArrayList<String> lines){
        ArrayList<String[]> tokenize_corpus = new ArrayList<>();
        for (String line : lines) {
            // removes punctuations
            line = line.replaceAll("\\p{Punct}","").toLowerCase().trim();

            tokenize_corpus.add(line.split(" "));
        }
        return tokenize_corpus;
    }
    
    public static HashMap<String, Double> words_count(String filename){
        // Function for the question 1
        ArrayList<String> lines = WordsCount.read_file(filename);
        ArrayList<String[]> lines_tokenized = WordsCount.tokenize(lines);
        HashMap<String, Double> words_count = new HashMap<>();

        for (String[] line_tokenized: lines_tokenized) {
            for (String word:line_tokenized) {
                if (word.isEmpty() != true){
                    if (words_count.get(word) == null) {
                            words_count.put(word, 1.0);
                    } else {
                            words_count.put(word, words_count.get(word)+1);
                    }
                }
            }
        }
        return words_count;
    }
    
    public static HashMap<String, Double> sorted_map_by_numeric_value(HashMap<String, Double> hash_map){
        // Function for question 2
        HashMap<String, Double> sorted_map = hash_map
                                            .entrySet()
                                            .stream()
                                            .sorted(Collections.reverseOrder(Map.Entry.comparingByValue()))
                                            .collect(toMap(Map.Entry::getKey, Map.Entry::getValue, (e1, e2) -> e2,
                                                    LinkedHashMap::new));
        return sorted_map;
    }
    
    public static HashMap<String, Double> sort(HashMap<String, Double> hash_map){
        // Function for question 3
        HashMap<String, Double> sorted_map = hash_map
                                                .entrySet()
                                                .stream()
                                                .sorted(new Comparator<Entry<String, Double>>() {
                                                    @Override
                                                    public int compare(Entry<String, Double> e1, Entry<String, Double> e2) {
                                                        if (e1.getValue().equals(e2.getValue())) {
                                                                return e1.getKey().compareTo(e2.getKey());
                                                        } else {
                                                            return e2.getValue().compareTo(e1.getValue());
                                                        }
                                                    }
                                                }).collect(toMap(Map.Entry::getKey, Map.Entry::getValue, (e1, e2) -> e1,
                                                        LinkedHashMap::new));
        return sorted_map;
    }
    
    public static void print_map(Map<String, Double> hash_map) {
        for (Entry<String, Double> el:hash_map.entrySet()) {
                System.out.println(el.getKey() + " " + el.getValue());
        }
    }
    
    public static void print_map(Map<String, Double> hash_map, Integer n) {
        Integer count = 0;
        for (Entry<String, Double> el:hash_map.entrySet()) {
            if(count == n){
                break;
            } else {
              System.out.println(el.getKey() + " " + el.getValue());
                count++;
            }
        }
    }
    
    public static void main(String[] args){
        System.out.print("Hello Jupyter from java");
    }
}

__1. Premier comptage en séquentiel pur__

Implémentez un logiciel en java qui compte le nombre d’occurrences des mots d’un fichier d’entrée de manière non parallélisée (monothread, une seul thread), en utilisant un seul processeur.<br> 
Quelle structure de donnée est la plus pertinente pour stocker les résultats: List, HashMap ou HashSet ou une autre ? Pour quelle raison ? 

 

Testez votre programme avec un fichier d’entrée input.txt avec comme contenu: 
```sh 
Deer Beer River 
Car Car River 
Deer Car Beer
```

Résultat: 
```sh 
Deer 2 
Beer 2 
River 2 
Car 3
```

In [21]:
WordsCount.print_map(WordsCount.words_count("data/input.txt"));

deer 2.0
car 3.0
river 2.0
beer 2.0


__2. Premier tri en séquentiel pur__

Modifiez votre programme pour trier par nombre d'occurrences: 

Résultat: 
```sh
Car 3 
Deer 2 
Beer 2 
River 2
```

In [17]:
HashMap<String, Double> words_count = WordsCount.words_count("data/input.txt");
WordsCount.print_map(WordsCount.sorted_map_by_numeric_value(words_count));

car 3.0
deer 2.0
river 2.0
beer 2.0


__3. Deuxième tri alphabétique en séquentiel pur__

Modifiez le programme pour trier alphabétiquement pour les mots à égalité du nombre d’occurrences: 

Résultat: 
```sh
Car 3 
Beer 2 
Deer 2
River 2 
```

In [22]:
HashMap<String, Double> words_count = WordsCount.words_count("data/input.txt");
WordsCount.print_map(WordsCount.sort(words_count));

car 3.0
beer 2.0
deer 2.0
river 2.0


__4. Test du programme séquentiel sur le code forestier de Mayotte__

Testez ensuite votre programme avec le code forestier de Mayotte disponible sur github forestier_mayotte.txt :  
https://github.com/legifrance/Les-codes-en-vigueur 

Votre programme a-t-il fonctionné du premier coup ?

Vérifiez en ouvrant le fichier texte qu’il contient bien du texte et non du code HTML. 

Ne perdez pas de temps à corriger les éventuelles erreurs dues aux caractères spéciaux ou à des mots suspects ou illisibles (de toutes façons par la suite il y aura du chinois dans le texte).

Le programme à fonctionné du premier coup, cependant les caractères spéciaux et la ponctuation faussaient les résultats. Comme demandé dans l'énoncé, je n'ai pas passé beaucoup de temps sur le nettoyage des données. J'ai seulement ajouté une regex spéciale Java afin de supprimer la ponctuation.

In [10]:
HashMap<String, Double> words_count = WordsCount.words_count("data/forestier_mayotte.txt");
WordsCount.print_map(WordsCount.sort(words_count), 15);

de 12.0
biens 8.0
ou 8.0
forestier 6.0
code 5.0
des 5.0
partie 5.0
agroforestiers 4.0
aux 4.0
communes 4.0
dispositions 4.0
forestiers 4.0
gestion 4.0
le 4.0
les 4.0


__5. Les 50 mots du code de la déontologie de la police nationale__

Testez votre programme avec le code de déontologie de la police nationale disponible sur github deontologie_police_nationale.txt : https://github.com/legifrance/Les-codes-en-vigueur 

De même ne perdez pas de temps à filtrer les caractères spéciaux ou autres mots bizarres. Pourquoi ? Car nous travaillerons ensuite sur des textes en chinois, japonais, arabe et d’autres langues. Si vous implémentez une étape de filtrage ici en français elle ne servira à rien par la suite. Quels sont les 5 premiers mots (qui ressemblent à des mots) parmi les 50 premiers de la liste triée résultat ? Gardez la réponse pour l’intégrer au rapport.

Les 5 premiers mots parmi les 50 premiers de la liste triée sont "de, la, police, et, des"

In [11]:
HashMap<String, Double> words_count = WordsCount.words_count("data/deontologie_police_nationale.txt");
WordsCount.print_map(WordsCount.sort(words_count), 50);

de 98.0
la 51.0
police 38.0
et 36.0
des 33.0
le 27.0
à 25.0
les 24.0
article 20.0
nationale 20.0
↬ 19.0
en 13.0
est 13.0
titre 13.0
ou 12.0
qui 11.0
fonctionnaires 10.0
lautorité 10.0
aux 9.0
code 9.0
fonctionnaire 9.0
par 9.0
commandement 8.0
du 8.0
leur 8.0
ses 8.0
au 7.0
devoirs 7.0
déontologie 7.0
il 7.0
ne 7.0
a 6.0
dans 6.0
cas 5.0
faire 5.0
lordre 5.0
ordres 5.0
pour 5.0
sa 5.0
se 5.0
si 5.0
sont 5.0
tout 5.0
doit 4.0
droits 4.0
elle 4.0
exécution 4.0
missions 4.0
pas 4.0
personne 4.0


__6. Les 50 mots du code du domaine public fluvial__

Testez votre programme avec le code du domaine public fluvial domaine_public_fluvial.txt. 

Quels sont les 5 premiers mots (qui ressemblent à des mots) parmi les 50 premiers de la liste triée résultat ?  Gardez la réponse pour l’intégrer au rapport.

Les 5 premiers mots parmi les 50 premiers de la liste triée sont "de, le, la, du, et"

In [12]:
HashMap<String, Double> words_count = WordsCount.words_count("data/domaine_public_fluvial.txt");
WordsCount.print_map(WordsCount.sort(words_count), 50);

de 630.0
le 429.0
la 370.0
du 347.0
et 295.0
les 240.0
des 222.0
à 209.0
est 173.0
dans 150.0
par 124.0
sur 123.0
ou 120.0
en 109.0
article 107.0
bateau 106.0
↬ 103.0
au 97.0
un 79.0
pour 73.0
tribunal 72.0
lieu 69.0
larticle 68.0
aux 66.0
il 66.0
dimmatriculation 64.0
qui 54.0
bureau 50.0
code 48.0
titre 48.0
navigation 46.0
que 46.0
dun 44.0
bateaux 43.0
où 43.0
son 43.0
domicile 42.0
se 41.0
créanciers 40.0
juge 38.0
a 37.0
propriétaire 37.0
commerce 36.0
saisie 36.0
intérieure 35.0
délai 34.0
nom 34.0
prix 34.0
sil 33.0
cas 32.0


__7. Les 50 mots du code de la santé publique__

Testez votre programme avec le code de la santé publique sante_publique.txt. 

Quels sont les 5 premiers mots (qui ressemblent à des mots) parmi les 50 premiers de la liste triée résultat ?  Gardez la réponse pour l’intégrer au rapport. 

Les 5 premiers mots parmi les 50 premiers de la liste triée sont "de, le, des, à, et"

In [13]:
HashMap<String, Double> words_count = WordsCount.words_count("data/sante_publique.txt");
WordsCount.print_map(WordsCount.sort(words_count), 50);

de 190889.0
la 82599.0
des 67813.0
à 65546.0
et 62702.0
les 62474.0
le 56800.0
du 48514.0
ou 40848.0
en 31742.0
par 30718.0
au 25730.0
dans 25104.0
article 23844.0
larticle 22998.0
↬ 21810.0
est 21792.0
un 20536.0
santé 20197.0
l 18762.0
pour 16876.0
aux 16144.0
sont 15048.0
une 13868.0
sur 12624.0
que 11348.0
r 11236.0
qui 10150.0
dun 9672.0
peut 8608.0
directeur 8604.0
lagence 8284.0
conditions 8094.0
être 8070.0
cas 8040.0
ne 7714.0
a 7622.0
conseil 7560.0
dune 7374.0
son 7298.0
général 6912.0
il 6666.0
ce 6352.0
ces 6188.0
1° 6114.0
dispositions 5926.0
2° 5872.0
leur 5766.0
pas 5702.0
sécurité 5282.0


### <font color="red">Etape 4</font> : Lancer des programmes java à distance manuellement.

__Un premier programme SLAVE sous Eclipse__

Faire un programme Java nommé “SLAVE” qui calcule 3+5, affiche le résultat, sous Eclipse (Pour lancer Eclipse: Menu applications>développement), lancer ce programme dans Eclipse (flèche verte “exécuter”)

In [1]:
public class Slave {

    public static void main() throws InterruptedException {
        System.out.println(3 + 5);
    }
}

In [26]:
new Slave().main()

8


__1. Exportation en JAR__

Exporter le programme Java en slave.jar exécutable (Java ARchive dite Runnable) sous Eclipse. Attention de bien vérifier que le JAR est de type “Runnable”/”exécutable”.

Le fichier `slave.jar` se trouve dans dossier `hadoop-from-scratch/jar`

__2. Exécution sur disque dur local__

Créez le répertoire `/tmp/<votre nom d’utilisateur>/`
Copiez  slave.jar exécutable dans le répertoire `/tmp/<votre nom d’utilisateur>/`
Testez et Lancer le slave.jar en ligne de commande sur votre ordinateur local.

__Important__ : Veuillez activer le kernel python3

In [11]:
user = !echo $USER
choice = input('Voulez vous lancer la création du dossier /tmp/{} y/[n]'.format(user[0]))
if choice == 'y':
    !if test ! -d /tmp/$USER; then mkdir -p /tmp/$USER; fi

Voulez vous lancer la création du dossier /tmp/axel y/[n] y


In [8]:
!cp jar/slave.jar /tmp/$USER && java -jar /tmp/$USER/slave.jar

8


__3. Copie du JAR et exécution distante__

Depuis la machine A contenant `/tmp/<votre nom d’utilisateur>/slave.jar` 
Créez à distance sur la machine B (s’il n’existe pas) un répertoire `/tmp/<votre nom d’utilisateur>/`
Copiez slave.jar sur la machine B dans le répertoire `/tmp/<votre nom d’utilisateur>/`
Exécutez à distance (depuis A sur la machine B) le slave.jar.
Quelle est la commande tapée pour effectuer cette dernière action ?

In [50]:
# Veuillez entrer nom d'une machine disponible sur le réseau et l'utilisateur
# (la connexion sans mot de passe via ssh doit être configurée)
remote_host = "tp-tests.enst.fr"
remote_user = "acamara"

!scp jar/slave.jar $remote_user@$remote_host:/tmp/$remote_user
!ssh $remote_user@$remote_host java -jar /tmp/$remote_user/slave.jar

slave.jar                                     100%   10KB   1.0MB/s   00:00    
8


__Important__ : Veuillez activer le kernel java

### <font color="red">Etape 5</font> : Lancer des programmes en ligne de commande depuis java et afficher la sortie standard et la sortie d’erreur.

__1. Un programme MASTER java qui lance un autre programme en ligne de commande!__

Ecrire un programme java nommé “MASTER” qui lance la commande suivante en utilisant ProcessBuilder:
```sh
ls -al /tmp
```
(vous pouvez aussi tester cette commande dans un terminal avant)
Récupérer et afficher la sortie de cette commande.
Vous devez utiliser ProcessBuilder de cette façon:

```java
ProcessBuilder pb = new ProcessBuilder("ls", “-al”, “/tmp”);
```

In [18]:
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;

public class Master {
    public static void run_shell_command(String[] command) {
        ProcessBuilder pb = new ProcessBuilder(command);

        try {
            Process process = pb.start();
            BufferedReader reader_input = new BufferedReader(new InputStreamReader(process.getInputStream()));

            String line;
            while ((line = reader_input.readLine()) != null) {
                System.out.println(line);
            }

        } catch (IOException e) {
            e.printStackTrace();
        }
    }

    public static void main(String[] args) throws IOException {
        Master.run_shell_command(args[0].split(" "));
    }
}

In [20]:
new Master().main(new String[]{"ls -al /tmp"});

total 648
drwxrwxrwt 17 root root   4096 Oct 13 16:39 .
drwxr-xr-x 23 root root   4096 Oct 10 13:26 ..
drwxrwxr-x  2 axel axel   4096 Oct 13 11:59 acamara
-rw-rw-r--  1 axel axel    405 Oct  7 23:15 gradle-worker-classpath1054293946014425259txt
-rw-r--r--  1 1000 1000    405 Sep 27 16:32 gradle-worker-classpath10796728312708717652txt
-rw-r--r--  1 1000 1000    405 Sep 27 17:56 gradle-worker-classpath16526685033490323404txt
-rw-r--r--  1 1000 1000    405 Sep 27 16:28 gradle-worker-classpath17730799258997419394txt
drwxr-xr-x  2 1000 1000   4096 Oct  3 15:42 hsperfdata_aksl
drwxr-xr-x  2 axel axel   4096 Oct 13 16:32 hsperfdata_axel
drwxr-xr-x  2 root root   4096 Sep 27 15:28 hsperfdata_root
-rw-rw-r--  1 axel axel   7273 Oct 12 16:06 kotlin-daemon.2019-10-12.14-52-46-710.00.log
-rw-rw-r--  1 axel axel   1328 Oct 12 16:56 kotlin-daemon.2019-10-12.16-56-18-013.00.log
-rw-rw-r--  1 axel axel   3451 Oct 12 17:08 kotlin-daemon.2019-10-12.16-59-57-526.00.log
-rw-rw-r--  1 axel axel  27300 Oct 

__2. Un programme MASTER java qui gère les erreurs de lancement d’un autre programme en ligne de commande.__

Modifiez votre programme”MASTER” pour qu’il affiche la sortie d’erreur en cas d’erreur lors de l’exécution de la commande. Testez la sortie d’erreur avec une commande qui échoue avec un sortie d’erreur. Essayez par exemple d’exécuter la commande “ls /jesuisunhero”.
Explications: si on tape la commande “ls /jesuiunhero”, le dossier /jesuisunhero n’existant pas, on aura une erreur de type “impossible d’accéder à /jesuisunhero: aucun fichier ou dossier de ce type.” qui s’affiche dans la sortie d’erreur. En effet, il y a deux sorties: les sorties standards (sans erreur) et les sorties d’erreurs.
Vous devez utiliser ProcessBuilder de cette façon:
```java
ProcessBuilder pb = new ProcessBuilder("ls", "/jesuisunhero”);
```
Sur `pb`, vous pouvez récupérer le flux de la sortie standard et le flux de la sortie d’erreur.

Afin de gérer les éventuelles erreurs lors du lancement du processus, nous pouvons utiliser la méthode `getErrorStream` de l'objet `Process`. Cette méthode revoie le flux de messages d'erreur du processus. 

In [1]:
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;

public class Master {
    public static void run_shell_command(String[] command) {
        ProcessBuilder pb = new ProcessBuilder(command);

        try {
            Process process = pb.start();
            BufferedReader reader_input = new BufferedReader(new InputStreamReader(process.getInputStream()));

            String line;
            while ((line = reader_input.readLine()) != null) {
                System.out.println(line);
            }
            
            BufferedReader reader_error = new BufferedReader(new InputStreamReader(process.getErrorStream()));

            while ((line = reader_error.readLine()) != null) {
                System.out.println(line);
            }

        } catch (IOException e) {
            e.printStackTrace();
        }
    }

    public static void main(String[] args) throws IOException {
        Master.run_shell_command(args[0].split(" "));
    }
}

In [2]:
new Master().main(new String[]{"ls -al /fr"});

ls: cannot access '/fr': No such file or directory


__3. Un programme MASTER java qui lance un slave.jar en ligne de commande.__

Modifiez votre programme “MASTER” pour qu’il lance “SLAVE”, c’est à dire slave.jar situé sur la même machine que “MASTER” dans le dossier 
```sh
/tmp/<votre nom d’utilisateur>/slave.jar
```

In [4]:
new Master().main(new String[]{"java -jar jar/slave.jar"});

8


### <font color="red">Etape 6</font> : Gérer les timeout du MASTER.

__1. Un SLAVE qui simule un calcul de 10 secondes.__

Modifiez votre programme SLAVE pour qu’il simule une attente de 10 secondes avant d’afficher le résultat du calcul 3+5. Pour cela utilisez
`Thread.sleep(10000);` 
Vérifiez le bon fonctionnement du SLAVE et constatez qu’il y a 10 secondes entre le démarrage du SLAVE et l’affichage du résultat. Attention de ne rien afficher avant les 10 secondes pour que la question suivante fonctionne correctement.
Générez de nouveau le slave.jar. Copiez-le en écrasant le slave.jar dans le dossier `/tmp/<votre nom d’utilisateur>/slave.jar`
Testez le lancement à partir de MASTER.

In [2]:
public class Slave {

    public static void main() throws InterruptedException {
        Thread.sleep(10000);
        System.out.println(3 + 5);
    }
}

In [3]:
new Slave().main()

8


__2. Gérer les timeout au niveau du MASTER__

Modifier le MASTER pour qu’il attende que quelque chose soit écrit dans la sortie standard (sans erreur) ou dans la sortie d’erreurs du SLAVE pendant un certain temps maximum. Au bout du temps imparti le MASTER considère un timeout.
Il arrête les éventuels threads (si vous utilisez des threads - non obligatoire) s’occupant des sorties et/ou stoppe le processus créé avec ProcessBuilder.

Vous devrez vérifier les TESTs suivants:
- TEST1 : Testez le bon fonctionnement du timeout en lançant le SLAVE avec un timeout de 2 secondes sur les sorties (standard et d’erreur). Le timeout étant plus court (au niveau du MASTER 2 secondes) que le temps de calcul du SLAVE (10 secondes), le MASTER doit arrêter les éventuels threads (si vous en utilisez) et le processus.
- TEST 2: Testez ensuite avec un timeout de 15 secondes, il ne devrait pas y avoir de timeout.
- TEST 3: Testez ensuite en changeant le SLAVE pour qu’il écrive non plus dans la sortie standard (sans erreur) mais dans la sortie d’erreur. Pour cela, remplacez dans le Slave les System.out.print… par System.err.print…

In [ ]:
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStream;
import java.io.InputStreamReader;
import java.util.Arrays;
import java.util.concurrent.LinkedBlockingQueue;
import java.util.concurrent.TimeUnit;

public class Master {

    static class ProcessLauncher {
        Process process;
        Integer timeout;
        String[] command;
        boolean error = false;
        ThreadReaderStream input_stream;
        ThreadReaderStream error_stream;

        ProcessLauncher(String command, Integer timeout) {
            this.timeout = timeout;
            this.command = command.split(" ");
            ProcessBuilder builder = new ProcessBuilder(this.command);
            try {
                this.process = builder.start();
            } catch (IOException e1) {
                e1.printStackTrace();
            }
            assert process != null;
            input_stream = new ThreadReaderStream(process.getInputStream());
            error_stream = new ThreadReaderStream(process.getErrorStream());
            input_stream.start();
            error_stream.start();
        }

        boolean launch_process() throws InterruptedException {
            String line;
            while (((line = input_stream.queue.poll(this.timeout, TimeUnit.SECONDS)) != null)) {
                System.out.print(line + "\n");
            }
            while (((line = error_stream.queue.poll()) != null)) {
                System.out.print(line + "\n");
                error = true;
            }
            return !error;
        }
    }

    static class ThreadReaderStream extends Thread {
        LinkedBlockingQueue<String> queue = new LinkedBlockingQueue<>();
        BufferedReader reader;

        ThreadReaderStream(InputStream stream) {
            this.reader = new BufferedReader(new InputStreamReader(stream));
        }

        @Override
        public void run() {
            try {
                String line;
                while ((line = reader.readLine()) != null) {
                    queue.put(line);
                }
            } catch (IOException | InterruptedException e) {
                e.printStackTrace();
            }
        }
    }


    public static void main(String[] args) throws InterruptedException {
        // Deploy the
        String[] pc = args;
        // Une facon de lancer les threads, sans attendre de retour de leur part:
        Arrays.asList(pc).parallelStream()
                .forEach(command -> {
                    try {
                        new ProcessLauncher(command, 2).launch_process();
                    } catch (InterruptedException e) {
                        e.printStackTrace();
                    }});
    }
}

### <font color="red">Etape 7</font> : Déployer automatiquement le programme SLAVE sur un ensemble de machines.

__1. Un programme DEPLOY : Test de connection SSH multiple__

Créer un fichier texte à la main contenant : les adresses IP et/ou les noms des machines que nous voulons utiliser pour notre système réparti (par exemple toutes les machines de cette salle de TP), avec un nom ou une IP par ligne dans le fichier.
Créer un nouveau programme java DEPLOY qui lit ce fichier ligne par ligne et teste si la connection SSH fonctionne bien sur chacune des machines. Attention, il s’agit bien d’un nouveau programme qui est séparé de MASTER ou SLAVE, vous ne l’exécuterez qu’en cas de mise à jour du SLAVE. Ceci permet de vérifier qu’une machine n’est pas éteinte ou qu’il y a un problème de connection (par exemple).
Pour vérifier que la connection fonctionne bien, vous pouvez faire afficher le nom de la machine distante (en exécutant la commande hostname à distance) et vérifier que l’affichage a effectivement lieu, sans erreurs. Réutilisez des parties de codes de la cinquième étape.
Votre programme DEPLOY lance-t-il les connections de manière séquentielle (les unes après les autres) ou de manière parallèle?